<font size=5>将手写数字总图分割为各个数字小图

In [2]:
import cv2
import numpy as np
import os
import shutil

In [132]:
pic_path = 'C:/Users/liu_w/Desktop/num_general.jpg' # 分割的图片的位置
pic_target = 'C:/Users/liu_w/Desktop/num_seg/' # 分割后的图片保存的文件夹
if not os.path.exists(pic_target):  #判断是否存在文件夹如果不存在则创建为文件夹
    os.makedirs(pic_target)
#要分割后的尺寸
#原图为2599×3637，14行×10列
cut_width = 2599//10
cut_length = 3637//14
# 读取要分割的图片，以及其尺寸等数据
picture = cv2.imread(pic_path)
(width, length, depth) = picture.shape
# 预处理生成0矩阵
pic = np.zeros((cut_width, cut_length, depth))
# 计算可以划分的横纵的个数
num_width = int(width / cut_width)
num_length = int(length / cut_length)
# for循环迭代生成
for i in range(0, num_width):
    for j in range(0, num_length):
        pic = picture[i*cut_width : (i+1)*cut_width, j*cut_length : (j+1)*cut_length, :]      
        result_path = pic_target + '{}+{}.png'.format(j, i) #将列号置前，以便后续整理图片至对应文件夹
        cv2.imwrite(result_path, pic)
 
print("done!!!")

done!!!


<font size=5>擦除分割好的小图边缘框线

In [133]:
rootdir = 'C:/Users/liu_w/Desktop/num_seg/'
list = os.listdir (rootdir)
for i in range(0, len(list)) :
    picture = cv2. imread("C:/Users/liu_w/Desktop/num_seg/"+list[i])
    (w, h, g)=picture.shape
    #print (w,h)
    #每边各裁剪20个像素
    dst = picture[20:w-20, 20:h-20]
    cv2. imwrite ("C:/Users/liu_w/Desktop/num_seg/"+list[i],dst) # 写入图片

print("done!!!")

done!!!


<font size=5>数据增强

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
rotation_range=30,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.1,
horizontal_flip=False,
brightness_range=[0.1, 2],
fill_mode='nearest')
 
trains = os.listdir("C:/Users/liu_w/Desktop/num_seg/")
print(len(trains))

for index,file in enumerate(trains):
    img = load_img("C:/Users/liu_w/Desktop/num_seg/" + file)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    
    i = 0
    for batch in datagen.flow(x, batch_size=10,
        save_to_dir='C:/Users/liu_w/Desktop/num_augemented', save_prefix=os.path.basename(file), save_format='png'):
        i += 1
 
        if i > 40:
             break


140


随机旋转

In [142]:
from math import *
import random

rootdir = 'C:/Users/liu_w/Desktop/num_seg/'
list = os.listdir (rootdir)

for i in range(0, len(list)) :
    img = cv2. imread('C:/Users/liu_w/Desktop/num_seg/'+list[i])
    w, h, _ = img.shape
    
    #每张图片在（-30°~30°）范围内随机旋转40次
    for j in range (40):
        angle = random.randint(-30, 30) 
        M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
        
        #去除黑边
        width_new = int(h * fabs(sin(radians(angle))) + w * fabs(cos(radians(angle))))
        height_new = int(w * fabs(sin(radians(angle))) + h * fabs(cos(radians(angle))))
        
        M[0, 2] += (height_new - h) / 2
        M[1, 2] += (width_new - w) / 2
        
        new_img = cv2.warpAffine(img, M, (w, h), borderValue=(255, 255, 255))
        result_path = 'C:/Users/liu_w/Desktop/num_seg_rotate/' +list[i] + '_r{}.png'.format(angle)
        cv2.imwrite(result_path,new_img)
        
    
print("done!!!")

done!!!


<font size=5>转换为黑底白字，并将分辨率改为28×28

In [15]:
rootdir = 'C:/Users/liu_w/Desktop/num_seg/'
list = os.listdir (rootdir)
for i in range(0, len(list)) :
    img = cv2. imread(rootdir + list[i])
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #转换为灰度
    w, h = img.shape[:2]
    imgInv = np.empty((w, h), np.uint8)
    for j in range(w):
        for k in range(h):
            imgInv[j][k] = 255 - imgGray[j][k]
    #修改为28×28像素
    picture = cv2.resize(imgInv, (28, 28))
    cv2. imwrite ("C:/Users/liu_w/Desktop/raw_dataset/"+list[i], picture) # 写入图片
    
rootdir = 'C:/Users/liu_w/Desktop/num_augemented/'
list = os.listdir (rootdir)
for i in range(0, len(list)) :
    img = cv2. imread(rootdir + list[i])
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #转换为灰度
    w, h = img.shape[:2]
    imgInv = np.empty((w, h), np.uint8)
    for j in range(w):
        for k in range(h):
            imgInv[j][k] = 255 - imgGray[j][k]
    #修改为28×28像素
    picture = cv2.resize(imgInv, (28, 28))
    cv2. imwrite ("C:/Users/liu_w/Desktop/raw_dataset/"+list[i], picture) # 写入图片

print("done!!!")

done!!!


<font size=5>整理图片进对应文件夹

In [16]:
path = "C:/Users/liu_w/Desktop/raw_dataset/"
target_path = "C:/Users/liu_w/Desktop/MNISTlw/"
print('图片所在目录：'+ path)

file_list = os.listdir(path)
print('当前图片数：' , len(file_list))

#在目标文件夹新建子文件夹
for i in range(10):
    os.mkdir(target_path + str(i))
    print('创建文件夹' + str(i))

for file in file_list:
    filename = os.path.basename(file)
    name = filename.split('+')
    #print(name[0])

    shutil.copy(path+'//'+filename , target_path+'/'+name[0])


图片所在目录：C:/Users/liu_w/Desktop/raw_dataset/
当前图片数： 5865
创建文件夹0
创建文件夹1
创建文件夹2
创建文件夹3
创建文件夹4
创建文件夹5
创建文件夹6
创建文件夹7
创建文件夹8
创建文件夹9
